In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import datetime

from dateutil.parser import parse

In [3]:
bike_path = os.path.join('..', '..', 'data', 'raw', '2017_Toronto_Bikeshare.csv')
bike_path

'../../data/raw/2017_Toronto_Bikeshare.csv'

In [4]:
df_raw = pd.read_csv(bike_path)

/Users/brycehaley/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (0,4,6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Check size and initial data types

In [5]:
df_OG = df_raw.copy()
df_raw.shape[0]

1492370

In [6]:
df_raw.dtypes

trip_id                   object
trip_start_time           object
trip_stop_time            object
trip_duration_seconds    float64
from_station_id           object
from_station_name         object
to_station_id             object
to_station_name           object
user_type                 object
dtype: object

In [7]:
df_raw.loc[2802, 'trip_duration_seconds']

434.0

# remove dead last row

In [7]:
last_row = len(df_raw) - 1
df_raw = df_raw.drop(df_raw.index[last_row])
print(df_raw.head(50))
print(df_raw.tail(50))

   trip_id trip_start_time trip_stop_time  trip_duration_seconds  \
0   712382   1/1/2017 0:00  1/1/2017 0:03                  223.0   
1   712383   1/1/2017 0:00  1/1/2017 0:05                  279.0   
2   712384   1/1/2017 0:05  1/1/2017 0:29                 1394.0   
3   712385   1/1/2017 0:07  1/1/2017 0:21                  826.0   
4   712386   1/1/2017 0:08  1/1/2017 0:12                  279.0   
5   712387   1/1/2017 0:12  1/1/2017 0:22                  571.0   
6   712388   1/1/2017 0:15  1/1/2017 0:33                 1084.0   
7   712389   1/1/2017 0:15  1/1/2017 0:21                  307.0   
8   712390   1/1/2017 0:20  1/1/2017 0:31                  647.0   
9   712391   1/1/2017 0:23  1/1/2017 0:27                  217.0   
10  712392   1/1/2017 0:28  1/1/2017 0:36                  467.0   
11  712393   1/1/2017 0:28  1/1/2017 0:43                  880.0   
12  712394   1/1/2017 0:31  1/1/2017 0:35                  228.0   
13  712395   1/1/2017 0:33  1/1/2017 0:39       

# check date validity

In [ ]:
def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

In [ ]:
df_raw['stop_flag'] = (df_raw['trip_stop_time'].apply(is_date) & df_raw['trip_stop_time'].apply(is_date))

In [ ]:
df_raw['stop_flag'].value_counts()

In [ ]:
df_raw[df_raw['stop_flag'] == False]

In [ ]:
df_raw = df_raw.drop( df_raw[df_raw['stop_flag'] == False].index)

# Check date order

In [ ]:
start_date_list = df_raw['trip_start_time'].values
start_date_list

In [ ]:
pd.to_datetime('12/31/17', dayfirst = True)

In [ ]:
df_raw['trip_start_time'] = pd.to_datetime(df_raw['trip_start_time'])
df_raw['trip_stop_time'] = pd.to_datetime(df_raw['trip_stop_time'])

In [ ]:
print('shape:', df_raw.shape)
print(df_raw.dtypes)
df_ttimes = df_raw

# Check trip times

In [ ]:
df_ttimes['check_time'] = (df_ttimes['trip_stop_time'] - df_ttimes['trip_start_time']) / np.timedelta64(1,'s')
df_ttimes.head()

In [ ]:
df_ttimes['time_delta'] = (df_ttimes['check_time'] - df_ttimes['trip_duration_seconds'])
df_ttimes.head()

In [ ]:
df_ttimes['time_flag'] = df_ttimes['time_delta'] > 500
df_ttimes.head()

In [ ]:
df_raw['time_flag'].value_counts()

# Examine bad times

In [ ]:
df_badtimes = df_ttimes[ df_ttimes['time_flag'] == True].copy()
df_badtimes

# Find when Date format changes

In [ ]:
df_dates = df_ttimes[['trip_start_time', 'trip_stop_time']].copy()
df_dates.head()

In [ ]:
df_dates['year_flag'] = (df_dates['trip_start_time'].dt.year == 2017) & (df_dates['trip_stop_time'].dt.year == 2017) 

In [ ]:
df_dates['year_flag'].value_counts()

In [ ]:
df_bad_dates = df_dates[ df_dates['year_flag'] == False].copy()
df_good_dates = df_dates[ df_dates['year_flag'] == True].copy()
df_bad_dates['start_day_17_flag'] = df_bad_dates['trip_start_time'].dt.day == 17
df_bad_dates['stop_day_17_flag'] = df_bad_dates['trip_stop_time'].dt.day == 17


print(df_bad_dates['start_day_17_flag'].value_counts())
print(df_bad_dates['stop_day_17_flag'].value_counts())

In [ ]:
df_bad_dates

In [ ]:
df_good_dates

In [ ]:
def date_mutator(row):
    
    return pd.Series({
        'trip_start_time': row['trip_start_time'],
        'trip_start_time': row['trip_start_time']
    })

In [ ]:
print('shape:', df_raw.shape)
print(df_raw.dtypes)
df_ttimes = df_raw

In [ ]:
df_ttimes.dtypes

# Check stops

In [ ]:
df_raw['from_station_id'].unique()

In [ ]:
len(df_raw['from_station_id'].unique())

In [ ]:
bool(re.match(r'[0-9]{4}', df_raw['from_station_id'][458752]))

In [ ]:
def matcher(x: str) -> bool:
    if type(x) == str:
        return bool(re.match(r'[0-9]{4}', x))
    return False

In [ ]:
type(df_raw['from_station_id'][458752])

In [ ]:
df_raw['good_id'] = df_raw['from_station_id'].apply(type)
df_raw['good_id'] = np.where(df_raw['good_id'] == int, True, False)

In [ ]:
df_bad = df_raw[df_raw['good_id'] ==  False].copy()
df_good = df_raw[df_raw['good_id'] == True].copy()
df_bad.shape

In [ ]:
df_bad['good_id'] = df_bad['from_station_id'].apply(matcher)

In [ ]:
df_bad

In [ ]:
df_new = df_bad[df_bad['good_id'] == True].copy()
df_new2 = df_bad[df_bad['good_id'] == False].copy()

In [ ]:
df_new

In [ ]:
df_new2

In [ ]:
df_new['from_station_id'] = df_new['from_station_id'].astype('int')
df_new['to_station_id'] = df_new['to_station_id'].astype('int')
df_good['from_station_id'] = df_good['from_station_id'].astype('int')
df_good['to_station_id'] = df_good['to_station_id'].astype('int')

In [ ]:
df_new['trip_start_time'] = pd.to_datetime(df_new['trip_start_time'])

In [ ]:
df_new.dtypes

In [ ]:
df_good.dtypes

In [ ]:

start_time_list = df_OG['trip_start_time'].values
start_time_list = start_time_list[0:-1]
start_time_list.shape
df_OG.dtypes

In [ ]:
for x in range(len(start_time_list)):
    start_time_list[x] = start_time_list[x].split()

In [ ]:
start_time_list

In [ ]:
start_time_list.shape

In [ ]:
for x in range(len(start_time_list)):
    if len(start_time_list[x]) != 2:
        print('error: ', x, ";", len(start_time_list[x]))

In [ ]:
for x in range(len(start_time_list)):
    start_time_list[x][0] = start_time_list[x][0].split('/')
    #start_time_list[x, 1] = start_time_list[x, 1].split(':')

In [ ]:
start_time_list

In [ ]:
date_split_list = []
for x in range(len(start_time_list)):
    if len(start_time_list[x][0]) != 3:
       date_split_list.append(x)

In [ ]:
months = pd.Series(start_time_list[:][0][0])

In [ ]:
days = pd.Series(start_time_list[:][0][1])

In [ ]:
years = pd.Series(start_time_list[:][0][2])

In [ ]:
print(date_split_list[0])
print(len(date_split_list))

In [ ]:
df_raw[